In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import string
import omdb
from omdb import OMDBClient
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.metrics import r2_score,mean_squared_error

I would like to use a dataset to predict how many Oscars a given film will win based on a variety of qualities--including genre, directors, the month they were released, etc. I plan to use a Random Forest Regressor.

In [ ]:
client = omdb.OMDBClient(apikey=APIKEY)

First I import the IMDB dataset that I downloaded here: https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset?select=IMDb+movies.csv

Thank you stefanoleone992!

In [ ]:
movies = pd.read_csv("IMDB movies.csv")

As you can see here, the dataset consists of over 85,000 movies and has a variety of data on each one; including language, genres, directors, actors, and more. You may notice, however, there is a rather important piece of data missing--whether or not each film won an oscar. This gives me the opportunity to scrape that data myself. 

In [ ]:
movies.shape

To append the award data to this dataset, I am going to use the omdb API. Essentially, it's an unofficial IMDB api, but it works well. Since I have the IMDB id for each film, I will simply query the website with the id, and grab the "award" field of the data. I will clean it up later with regular expressions, but it will tell me how many Oscars each film has won once I do that.

In [ ]:
def getAward(imdbId):
    try:
        return client.imdbid(imdbId)["awards"]
    except:
        return "Not found"

In [ ]:
movies["awards"] = movies.iloc[:,0].apply(lambda x : getAward(x))

I would like to build a random forest regression to predict how many oscars a given film will win based on a variety of factors that I think could be good indicators of an award-winning movie. Genre and language are both aspects that I think would be important in this prediction. In fact, Parasite was the very first non-English film to win an Oscar for best picture.

However, the data--as it stands--is not properly formatted. Genre and language both currently exist as lists for each film. I am going to use binary encoding to give the regression model a way to quantify this categorical data.

First, I am going to use the strip function to remove the white space on either side of the languages and genres. 

In [ ]:
cleaned_genre = movies.set_index(['imdb_title_id'])["genre"].str.split(',',expand=True).stack()

In [ ]:
cleaned_lang = movies.set_index(['imdb_title_id'])["language"].str.split(',',expand=True).stack()

In [ ]:
cleaned_lang = cleaned_lang.apply(lambda x : x.strip())

In [ ]:
cleaned_genre = cleaned_genre.apply(lambda x : x.strip())

Now I will actually begin the binary encoding process, using the get_dummies function to turn these lists into categorical data. This creates a new dataframe just containing each film by its id and binary indications of whether or not it is in a certain language.

In [ ]:
lang_encoded = pd.get_dummies(cleaned_lang, prefix='l').groupby(level=0).sum().reset_index()

Now I will merge the encoded languages for each movie onto the original movie dataframe.

In [ ]:
movies = movies.merge(lang_encoded, left_on=["imdb_title_id"], right_on=["imdb_title_id"])

I will repeat this same process in order to binary encode the genre values.

In [ ]:
genre_encoded = pd.get_dummies(cleaned_genre, prefix='g').groupby(level=0).sum().reset_index()

In [ ]:
movies = movies.merge(genre_encoded, left_on=["imdb_title_id"], right_on=["imdb_title_id"])

Now I will take a look at the awards column. The simple webscraper I executed just pulled all of the awards that each film won, usually in sentence structure format (e.g. "Won 2 Oscars. Another 112 wins & 103 nominations.") Rather than slow down the webscraper by extracting the number of Oscars as it went, I opted to wait until now to use a regular expression to extract the number representing how many Oscars the film won. I put this in separate column "oscars_won"

Obviously this will yield NaN values for any films that did not contain the regular expression (and thus did not win Oscars), so I will go ahead and fill those NaN values with 0.

In [ ]:
movies["oscars_won"] = movies[movies["awards"].str.contains(r'Won [1-9]* Oscar', regex=True)]["awards"].str.extract('(\d+)')

In [ ]:
movies["oscars_won"] = movies["oscars_won"].fillna(0)

Since I was working with strings, the numbers from the regular expression extraction would also be strings, so I convert those to integers here.

In [ ]:
movies["oscars_won"] = movies["oscars_won"].astype(int)

And then I go ahead and drop the original "awards" column because I have extracted the data I need from it and it will serve no purpose.

In [ ]:
movies = movies.drop("awards", axis =1)

There is one movie that is oddly labeled with "TV Movie 2019" rather than an actual date published. Not only will this interfere with the conversion of dates to datetime objects, I'm also not particularly interested in having a TV movie mixed in with this dataset. So I will go ahead and just drop that row.

In [ ]:
tvMovieIndex = movies[movies["date_published"]=="TV Movie 2019"].index

In [ ]:
movies = movies.drop([tvMovieIndex[0], tvMovieIndex[0]])

Now I will convert the date published column--currently a string--to datetime objects. I would also like to use the month a movie came out rather than the date in order to categorize the films more neatly, so I will create a "months_published" column from the converted "date_published" column.

In [ ]:
movies["date_published"] = pd.to_datetime(movies["date_published"])

In [ ]:
movies["month_published"] = movies["date_published"].dt.month

In [ ]:
movies["country"] = movies["country"].fillna('None')

In [ ]:
movies["director"] = movies["director"].astype('category')

In [ ]:
movies["director_cat"] = movies["director"].cat.codes

In [ ]:
movies["American"] = movies["country"].str.contains("USA")

In [ ]:
movies["American"] = movies["American"].fillna(0)

In [ ]:
factors = ["duration", 'g_Action', 'g_Adult',
       'g_Adventure', 'g_Animation', 'g_Biography', 'g_Comedy', 'g_Crime',
       'g_Documentary', 'g_Drama', 'g_Family', 'g_Fantasy', 'g_Film-Noir',
       'g_History', 'g_Horror', 'g_Music', 'g_Musical', 'g_Mystery', 'g_News',
       'g_Reality-TV', 'g_Romance', 'g_Sci-Fi', 'g_Sport', 'g_Thriller',
       'g_War', 'g_Western', 'month_published'] + movies.columns[22:].tolist()

Below I will reset the index in order to prevent any errors

In [ ]:
movies.reset_index(drop=True)

I'll go ahead and create the train and the test subsets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(movies[factors], movies["oscars_won"], train_size=0.7,test_size=0.3, random_state=0)

I'm going to create the random forest regressor. Now, it may seem strange to use just 1 n estimator, but it provides similarily accurate results as using a higher number, and doesn't result in decimal points for the resulting predictions. That is, using a higher number would result in a given film receiving a prediction of 8.564554 or so Oscars which is impossible. By keeping it to just one, we avoid that and it does not become exceedingly more inaccurate.

In [ ]:
regressor = RandomForestRegressor(n_estimators=1, random_state=0)

Here I fit my dataset to the regressor, generate the y predictions, and then check the results using the root mean squared error. This turned out to be a low ~.02. So I can feel fairly confident that my predictions were correct.

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)

In [ ]:
rmse = np.sqrt(mse)

In [ ]:
rmse